In [1]:
from glob import glob
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Reshape
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

In [2]:
no_of_patients=25

In [3]:
train=[]
test=[]
for i in range(1,no_of_patients+1):
    files=glob('C:/Users/kotha/Desktop/matplot/eeg/files/S'+str(str(i).zfill(3))+'/*.edf')
    files=files[2:3]
    train+=files
train

['C:/Users/kotha/Desktop/matplot/eeg/files/S001\\S001R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S002\\S002R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S003\\S003R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S004\\S004R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S005\\S005R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S006\\S006R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S007\\S007R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S008\\S008R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S009\\S009R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S010\\S010R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S011\\S011R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S012\\S012R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S013\\S013R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S014\\S014R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S015\\S015R03.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S016\\S016R0

In [4]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_data=eeg_data[:8]
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
#     display(eeg_df)
    eeg_df = eeg_df.iloc[:15000]
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

In [5]:
def scale_dataset(dataframe):
    x=dataframe[dataframe.columns[:-1]].values
    y=dataframe[dataframe.columns[-1]].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)  
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid
display(xtrain)
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [7]:
tf.random.set_seed(1234)
model = Sequential()
#model.add(Reshape((64,1),input_shape=(64,)))
model.add(Reshape((64,1)))
model.add(Conv1D(32, kernel_size=11, activation='relu',input_shape=(300000,64,1)))
model.add(Conv1D(64, kernel_size=11, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, kernel_size=11, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(25, activation='softmax'))

In [8]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63,id
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028,0
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040,0
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037,0
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026,0
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,-0.000005,-0.000025,-0.000043,0.000008,-0.000020,-0.000024,-0.000028,-0.000019,-0.000024,-0.000022,...,-0.000007,-0.000014,-0.000004,-0.000009,-0.000003,0.000001,0.000001,-0.000004,0.000005,24
299996,-0.000013,-0.000029,-0.000046,0.000006,-0.000021,-0.000020,-0.000010,-0.000026,-0.000027,-0.000025,...,-0.000016,-0.000020,-0.000005,-0.000011,-0.000008,-0.000002,-0.000004,-0.000010,-0.000001,24
299997,-0.000021,-0.000029,-0.000045,0.000008,-0.000017,-0.000014,-0.000005,-0.000028,-0.000028,-0.000026,...,-0.000008,-0.000015,-0.000004,-0.000011,-0.000008,0.000000,-0.000003,-0.000012,0.000000,24
299998,-0.000003,-0.000022,-0.000039,0.000013,-0.000012,-0.000010,-0.000012,-0.000020,-0.000021,-0.000021,...,-0.000005,-0.000011,0.000001,-0.000005,0.000001,0.000004,-0.000001,-0.000008,0.000002,24


In [9]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
     metrics=['accuracy']
)

model.fit(
    x,y,epochs=30
)

Epoch 1/30
9375/9375 [==============================] - 152s 16ms/step - loss: 0.5676
Epoch 2/30
9375/9375 [==============================] - 152s 16ms/step - loss: 0.1694
Epoch 3/30
9375/9375 [==============================] - 129s 14ms/step - loss: 0.1164
Epoch 4/30
9375/9375 [==============================] - 111s 12ms/step - loss: 0.0955
Epoch 5/30
9375/9375 [==============================] - 115s 12ms/step - loss: 0.0834
Epoch 6/30
9375/9375 [==============================] - 118s 13ms/step - loss: 0.0748
Epoch 7/30
9375/9375 [==============================] - 135s 14ms/step - loss: 0.0678
Epoch 8/30
9375/9375 [==============================] - 131s 14ms/step - loss: 0.0655
Epoch 9/30
9375/9375 [==============================] - 128s 14ms/step - loss: 0.0612
Epoch 10/30
9375/9375 [==============================] - 125s 13ms/step - loss: 0.0616
Epoch 11/30
9375/9375 [==============================] - 125s 13ms/step - loss: 0.0586
Epoch 12/30
9375/9375 [=============================

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (32, 64, 1)               0         
                                                                 
 conv1d (Conv1D)             (32, 54, 32)              384       
                                                                 
 conv1d_1 (Conv1D)           (32, 44, 64)              22592     
                                                                 
 max_pooling1d (MaxPooling1D  (32, 22, 64)             0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (32, 12, 64)              45120     
                                                                 
 conv1d_3 (Conv1D)           (32, 2, 128)              90240     
                                                        

In [11]:
y_pred=model.predict(xt)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(yt,y_pred_classes))

2343/2343 [==============================] - 12s 5ms/step
              precision    recall  f1-score   support

           0       0.93      0.82      0.88      2999
           1       0.91      0.93      0.92      2999
           2       0.86      0.86      0.86      2999
           3       0.97      0.97      0.97      2999
           4       0.98      0.99      0.99      2999
           5       0.99      0.99      0.99      2999
           6       0.98      0.98      0.98      2999
           7       0.95      0.92      0.93      2999
           8       0.78      0.97      0.86      2999
           9       0.90      0.95      0.93      2999
          10       0.97      0.92      0.94      2999
          11       1.00      0.97      0.98      2999
          12       0.73      0.90      0.81      2999
          13       0.90      0.98      0.94      2999
          14       0.68      0.86      0.76      2999
          15       0.95      0.81      0.88      2999
          16       0.95

In [ ]:
y_pred=model.predict(xv)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(yv,y_pred_classes))

In [ ]:
y_pred=model.predict(x)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y,y_pred_classes))

In [ ]:
history = model.fit(x,y, epochs=100, validation_data=(xv,yv))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Epochs vs. Accuracy')
plt.show()

# Plotting epochs vs. loss
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Epochs vs. Loss')
plt.show()